# word2vec模式下的两个模型：CBOW和SkipGram

文章参考[如何通俗理解Word2Vec (23年修订版)](https://blog.csdn.net/v_JULY_v/article/details/102708459)


## 1. Bigram--用前一个单词预测后一个单词

假定整个词汇表的大小为$V$, 即全世界所有的词语总共有 $V$ 个，这 $V$ 个词语有自己的先后顺序,那么我们可以用one-hot向量来指示每一个词。对于最简单的例子，即输入输出都只有一个词，假如$x$代表前一个词的one-hot向量，$y$代表后一个词的one-hot向量，那么此时语言模型的作用就是用当前词 $x$ 预测它的下一个词 $y$.


<figure>
<img src= "https://i-blog.csdnimg.cn/blog_migrate/99293ab952cd52c28264d3552cd6efcd.png" width=600/>
</figure>

如上图,这个网络结构里涉及神经网络中经典的输入层、隐藏层、输出层。通过从输入层到隐藏层，或隐藏层到输出层的权重矩阵去向量化表示词的输入。学习迭代的是两个权重矩阵：可分别用$W_{V\times N}、W'_{N \times V}$ [当然，也可以分别简写为W、W'，此外，其中V: 词汇表长度，N: 隐层神经元个数，同时也是词向量维度] <p>

这里 $W$ 其实就是词向量矩阵，其中每一行代表一个词的词向量.而 $W'$ 是隐藏层到输出层的权重矩阵，其中每一列也可以看作额外的一种词向量。但这 $W'$ 的计算量巨大，所以训练时一般用这两种加速算法：
- Hierarchical softmax「简称HS，是softmax 的一种近似形式，本质是把 N 分类问题变成 log(N)次二分类，具体下文会介绍」
- 负采样 (即negative sampling，本质是预测总体类别的一个子集)


**注意.1**： 为什么$W'$ 的计算量巨大，请见[Xin Rong,(2014)](https://arxiv.org/abs/1411.2738)   <p>
**注意.2**：word2vec 本质上是一种降维操作——把词语从 one-hot encoder 形式的表示降维到 Word2vec 形式的表示



## 2. CBOW & Skip-Gram Model


- CBOW(Continuous Bag-of-Word)：以上下文词汇预测当前词，即用$ w_{t-2}, w_{t-1}, w_{t+1}, w_{t+2}$ 去预测 $ w_{t}$
- SkipGram：以当前词预测其上下文词汇，即用$w_{t}$ 去预测 $ w_{t-2}, w_{t-1}, w_{t+1}, w_{t+2}$


下图是[Xin Rong,(2014)](https://arxiv.org/abs/1411.2738)对两个模型的描述图。

<figure>
<img src= "https://i-blog.csdnimg.cn/blog_migrate/87dc43b16e34ac1dcb32d2f28ccfaf37.png" width=500/>
<img src= "https://i-blog.csdnimg.cn/blog_migrate/c6259b6768a68ae8035871f0e3afb68e.png" width=500/>
</figure>



### 2.1 CBOW的简单实例


CBOW模型训练的基本步骤包括：

- 将上下文词进行one-hot 表征作为模型的输入，其中词汇表的维度为V，上下文单词数量C
- 然后将所有上下文词汇的 one-hot 向量分别乘以输入层到隐层的权重矩阵W
- 将上一步得到的各个向量相加取平均作为隐藏层向量
- 将隐藏层向量乘以隐藏层到输出层的权重矩阵$W'$
- 将计算得到的向量做 softmax 激活处理得到 V 维的概率分布，取概率最大的索引作为预测的目标词



## Optimizing Computational Efficiency

To update the para-matrix $ {W_{N × V}}'$ we need to compute $y_{j}, e_{j}$ for every $j = 1,2,...,V$ per training instance, when $V$ is large, its very expensive. To solve this problem, an intuition is to **limit the number of output vectors that must be updated per training instance**. One elegant achieving this is **hierarchical softmax**; another approach is through **sampling**.